# NX-MIMOSA Intelligence Pipeline — Platform ID, ECM Detection, Intent Prediction

Demonstrates the intelligence overlay that turns kinematic tracks into **actionable threat assessments**.

Features:
- **Platform classification**: 111 types from kinematics alone (no IFF needed)
- **ECM detection**: Noise jamming, deception, DRFM, chaff
- **Intent prediction**: 16 behavior types with threat scoring
- **Composite threat**: Single 0-1 score for each track

In [ ]:
import numpy as np
import sys; sys.path.insert(0, '..')
from nx_mimosa import (
    IntelligencePipeline, ThreatLevel, IntentType, ECMStatus
)
print("Intelligence module loaded")

## 1. Fighter Ingress Scenario

In [ ]:
intel = IntelligencePipeline()

# Simulate a fighter closing at 280 m/s, 5000m altitude, mild maneuvering
for t in range(25):
    report = intel.assess(
        track_id=1,
        speed_ms=280, altitude_m=5000,
        heading_rad=0.1, g_load=2.0 + 0.5 * np.sin(t * 0.3),
        range_to_asset_m=80000 - t * 2000,
        snr_db=22, rcs=3.0, doppler_hz=560, nis=1.8,
    )

print("=== Fighter Ingress Report ===")
print(f"Platform: {report.platform.platform_type} ({report.platform.coarse_class})")
print(f"Speed class: {report.platform.speed_class}")
print(f"Platform confidence: {report.platform.confidence:.2f}")
print(f"ECM status: {report.ecm.status.value}")
print(f"Intent: {report.intent.primary_intent.value}")
print(f"Threat level: {report.intent.threat_level.value}")
print(f"Composite threat: {report.composite_threat:.2f}")

## 2. Transport Aircraft — Benign

In [ ]:
for t in range(25):
    report = intel.assess(
        track_id=2,
        speed_ms=230, altitude_m=11000,
        heading_rad=1.5, g_load=1.0,
        range_to_asset_m=150000,
        snr_db=18, rcs=40.0, doppler_hz=460, nis=1.2,
    )

print("=== Transport Report ===")
print(f"Platform: {report.platform.platform_type}")
print(f"Speed class: {report.platform.speed_class}")
print(f"Intent: {report.intent.primary_intent.value}")
print(f"Composite threat: {report.composite_threat:.2f}")
print()
print("Transport should have LOWER threat than fighter:")
r1 = intel.assess(track_id=1, speed_ms=280, altitude_m=5000,
                  heading_rad=0.1, g_load=2.0, range_to_asset_m=40000)
print(f"  Fighter threat:   {r1.composite_threat:.2f}")
print(f"  Transport threat: {report.composite_threat:.2f}")

## 3. Multi-Track Threat Board

In [ ]:
# Simulate 4 tracks with different profiles
scenarios = {
    100: {"name": "SAM Missile", "speed": 900, "alt": 12000, "g": 15, "range": 20000, "heading": 0.0},
    101: {"name": "Patrol Helo", "speed": 60, "alt": 500, "g": 1.2, "range": 30000, "heading": 0.5},
    102: {"name": "Stealth UAV", "speed": 180, "alt": 6000, "g": 1.5, "range": 45000, "heading": 0.2},
    103: {"name": "Airliner",    "speed": 250, "alt": 12000, "g": 1.0, "range": 200000, "heading": 2.0},
}

intel2 = IntelligencePipeline()
print(f"{'Track':>6s} | {'Scenario':>12s} | {'Platform':>20s} | {'Intent':>15s} | {'Threat':>7s}")
print("-" * 80)

for tid, s in scenarios.items():
    for _ in range(20):
        r = intel2.assess(
            track_id=tid, speed_ms=s["speed"], altitude_m=s["alt"],
            heading_rad=s["heading"], g_load=s["g"],
            range_to_asset_m=s["range"],
        )
    print(f"{tid:>6d} | {s['name']:>12s} | {r.platform.platform_type:>20s} | "
          f"{r.intent.primary_intent.value:>15s} | {r.composite_threat:>6.2f}")

## Key Takeaways

1. **No IFF required** — platform classification from kinematics alone
2. **Real-time capable** — pure NumPy, no ML inference overhead
3. **Hierarchical** — speed → class → fine type → intent → threat
4. **Per-track independent** — each track has its own intelligence state

Commercial versions include 111 platform types, 16 intent behaviors, 
and 4 ECM categories. Contact mladen@nexellum.com for licensing.